In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from DatasetHandler import DatasetHandler
from findpeaks import findpeaks
import findpeaks
from scipy import special
from tqdm import tqdm

In [ ]:
def enl(img):
    return (np.nanmean(img)**2)/(np.nanstd(img)**2)

In [ ]:
def reject_outliers(s1):
    p = np.percentile(s1, 95)
    s1 = np.clip(s1, 0.0, p)

    return s1
    
def min_max(s1):
    return (s1 - s1.min()) / (s1.max() - s1.min())

## Load real data

In [ ]:
import rasterio
#path = 'GRD_data/S1-lat_41_13694930422632_lon_14_755163002014147-2019-09-12.tif'
path = 'GRD_data/subset_0_of_S1A_IW_GRDH_1SDV_20211020T174045_20211020T174110_040206_04C337_6D4A_Orb_Bdr_Thermal_Cal_TC.tif'

with rasterio.open(path) as src:
    vv = src.read(1)
    vh = src.read(2)

intensity = vh

print(' Max', intensity.max(), '\n Min', intensity.min(), '\n Std', intensity.std(), '\n Mean', intensity.mean())

In [ ]:
intensity = reject_outliers(intensity)
print(' Max', intensity.max(), '\n Min', intensity.min(), '\n Std', intensity.std(), '\n Mean', intensity.mean())

In [ ]:
intensity = min_max(intensity)
print(' Max', intensity.max(), '\n Min', intensity.min(), '\n Std', intensity.std(), '\n Mean', intensity.mean())

## Plot data

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (20,8))
ax[0].imshow(intensity, cmap ='gray', vmin = 0, vmax = 1)

ax[1].hist(intensity.flatten(), 100)

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (20,8))
#ax.imshow(intensity[2000:3500, 5000:6000], cmap ='gray', vmin = 0, vmax = 1)
ax.imshow(intensity[1000:2500, 1000:2500], cmap ='gray', vmin = 0.0, vmax = 1)
ax.axis(False)


plt.show()

In [ ]:
plt.imsave('input.png', intensity[2000:3500, 5000:6000], cmap='gray')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (20,8))
ax.imshow(intensity[2000:2300, 5360:5770], cmap ='gray', vmin = 0, vmax = 1)
ax.axis(False)
plt.show()

In [ ]:
plt.imsave('input2.png', intensity[2000:2300, 5360:5770], cmap='gray')

# Proposed filter

In [ ]:
import time
from Model import CNNSpeckleFilter
from tensorflow.keras.models import load_model

N_LAYER = 12
IMG_SHAPE = (256,256,1)

speckle_filter = CNNSpeckleFilter(input_shape=IMG_SHAPE, n_layers=N_LAYER)
speckle_filter.model.load_weights('weights/new_model_12.h5')


In [ ]:
start_time = time.time()

a = speckle_filter.model.predict(intensity[np.newaxis, :IMG_SHAPE[0], :IMG_SHAPE[1], np.newaxis])

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
prediction = np.zeros(intensity.shape)

start_time = time.time()

for i in tqdm(range(intensity.shape[0]//IMG_SHAPE[0])):
    for j in range(intensity.shape[1]//IMG_SHAPE[1]):
        prediction[i*IMG_SHAPE[0]:(i+1)*IMG_SHAPE[0],j*IMG_SHAPE[1]:(j+1)*IMG_SHAPE[1]] = speckle_filter.model.predict(intensity[np.newaxis, i*IMG_SHAPE[0]:(i+1)*IMG_SHAPE[0],j*IMG_SHAPE[1]:(j+1)*IMG_SHAPE[1],np.newaxis])[0,...,0]
 
print("--- %s seconds ---" % (time.time() - start_time))
prediction = prediction[:(intensity.shape[0]//IMG_SHAPE[0])*IMG_SHAPE[0], :(intensity.shape[1]//IMG_SHAPE[1])*IMG_SHAPE[1]]

In [ ]:
prediction = min_max(prediction)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (20,8))
ax[0].imshow(prediction, cmap ='gray', vmin = 0, vmax = 1)
ax[0].axis(False)

ax[1].hist(prediction.flatten(), 100)

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (20,8))
ax.imshow(prediction[2000:3500, 5000:6000], cmap ='gray', vmin = 0.0, vmax = 1.0)
#ax.imshow(prediction[1000:2500, 1000:2500], cmap ='gray', vmin = 0.15, vmax = 0.75)

ax.axis(False)


plt.show()

In [ ]:
plt.imsave('proposed.png', prediction[2000:3500, 5000:6000], cmap='gray',  vmin = 0.0, vmax = 1.0)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (20,8))
ax.imshow(prediction[2000:2300, 5360:5770], cmap ='gray', vmax = 1)
ax.axis(False)
plt.show()

In [ ]:
plt.imsave('proposed2.png', prediction[2000:2300, 5360:5770], cmap='gray',  vmax = 1.1)

# SARBM3D

In [ ]:
import rasterio
# path = 'GRD_data/S1-lat_41_13694930422632_lon_14_755163002014147-2019-09-12.tif'
path = 'GRD_data/SARBM3D.tif'

with rasterio.open(path) as src:
    sarbm3d = src.read(1).astype(np.float64)


In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (20,8))
ax[0].imshow(sarbm3d, cmap ='gray', vmin = 0, vmax = 200)
ax[0].axis(False)

ax[1].hist(sarbm3d.flatten(), 100, [0,300])

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (20,8))
ax.imshow(sarbm3d[2000:3500, 5000:6000], cmap ='gray', vmin = 0, vmax = 200)
ax.axis(False)


plt.show()

In [ ]:
plt.imsave('SARBM3D.png', sarbm3d[2000:3500, 5000:6000], cmap='gray',  vmin = 0, vmax = 200)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (20,8))
ax.imshow(sarbm3d[2000:2300, 5360:5770], cmap ='gray', vmin = 0, vmax = 200)
ax.axis(False)
plt.show()

In [ ]:
plt.imsave('SARBM3D2.png', sarbm3d[2000:2300, 5360:5770], cmap='gray',  vmin = 0, vmax = 200)